# 函数准备

In [1]:
import os
import numpy as np
import pandas as pd
import json
from copy import deepcopy
import re
import random
from random import choice
random.seed(10)
# -*- coding:utf-8 -*
import eval
from mkdir import mk_dir
from bert_format import bset,test_qa
import spokentonumer

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 5)

for col_ab in ["age", "gender","num","evi"]:

    mk_dir("eval/evl_%s" % (col_ab))

    train_set = json.load(open("input/input_%s/train_set.json" \
        %(col_ab), "r"))
    train_qa = train_set['data'][0]["paragraphs"]

    re = 0
    for ttrain_qa,eval_qa in kf.split(train_qa):

        re = re + 1

        ttrain_set = {
            "data": [{
                'paragraphs': [train_qa[i] for i in ttrain_qa],
                'title': "LungB"
            }],
            "version": "LungB"
        }

        mk_dir("eval/evl_%s/input_%s" % (col_ab, re))
        mk_dir("eval/evl_%s/output_%s" % (col_ab, re))

        json.dump(ttrain_set, open("eval/evl_%s/input_%s/train_set.json" 
        % (col_ab, re), "w"))

        eval_q=[]
        eval_a={}
        for i in eval_qa:
            qq={
                'qas': [{
                    'id': train_qa[i]['qas'][0]['id'],
                    'question': train_qa[i]['qas'][0]['question'],
                    'answers':[]
                }],
                'context': train_qa[i]['context']
            }
            eval_q.append(qq)
            eval_a[train_qa[i]['qas'][0]['id']] = train_qa[i]['qas'][0]['answers']

        eval_qset = {
            "data": [{
                'paragraphs': eval_q,
                'title': "LungB"
            }],
            "version": "LungB"
        }

        json.dump(eval_qset, open("eval/evl_%s/input_%s/test_set.json" 
        % (col_ab, re), "w"))
        json.dump(eval_a, open("eval/evl_%s/input_%s/eval_set.json" 
        % (col_ab, re), "w"))

In [ ]:
col_ab =  "evi"
standard_0 = pd.DataFrame(np.zeros((0,1)), columns = ["golden"])
prediction_0 = pd.DataFrame(np.zeros((0,3)), columns = ["prediction","rate","cut"])
for re in range(1,11):
    eval_a = json.load(open("eval/evl_%s/input_%s/eval_set.json" % (col_ab, re), "r"))
    standard = pd.DataFrame(eval_a, index=["golden"]).T
    standard_0 = pd.concat((standard_0, standard), 0)
    prediction = json.load(open("eval/evl_%s/output_%s/predictions_.json" % (col_ab, re),'r'))
    prediction = pd.DataFrame(prediction, index=["prediction", "rate"]).T
    prediction["cut"] = 1
    for idx in prediction.index:
        prediction.loc[idx, "rate"] = prediction.loc[idx, "rate"][0]
        prediction.loc[idx, "cut"] = 1 - prediction.loc[idx, "rate"]
        if prediction.loc[idx, "rate"] < 0.85:
            prediction.loc[idx, "prediction"] = "no"
    prediction_0 = pd.concat((prediction_0, prediction),0)

fig_path = ("eval/evl_%s/output_%s" % (col_ab, re))
evaluate(standard_0, prediction_0, fig_path)

# 患者数量

In [ ]:
df = pd.read_excel("/home/data/t070224/Pubmed/Pubmed_Label/3.Orgnize/output/文献合并/human_label_pat.xlsx")
# df = df.loc[df["是否是致病菌"]=='yes', :]
df = df.fillna("none")
df['golden'] = df['样本量']
df["prediction"] = 'none'

for i in df.index:
    
    if df.loc[i,"Full"] != 'none':
        
        digit_list = spokentonumer.del_non_digit(df.loc[i, 'Full'])
        
        if (len(digit_list) > 0) & ("to" not in df.loc[i, 'Full'].lower()):
                
            clasn = digit_list[0]
            if int(clasn) == 1:
                clas = "1"
            elif int(clasn) in range(2,11):
                clas = "2-10"
            elif int(clasn) in range(11,101):
                clas = "11-100"
            elif int(clasn) > 100:
                clas = ">100"
            else:
                clas = 'none'
                
            df.loc[i, "prediction"] = clas

In [ ]:
spokentonumer.del_non_digit(df.loc[i, 'Full'])

In [17]:
eval.multiclass_score(df[["golden"]], df[["prediction"]])

结果: 0.4340659340659341 0.7550088075920977 0.4340659340659341 0.49338693424061786


(0.4340659340659341,
 0.7550088075920977,
 0.4340659340659341,
 0.49338693424061786)

# 患者年龄

In [18]:
df = pd.read_excel("/home/data/t070224/Pubmed/Pubmed_Label/3.Orgnize/output/文献合并/human_label_pat.xlsx")
df = df.loc[df["是否是致病菌"]=='yes', :]
df = df.fillna("none")
df['golden'] = df['年龄']
df["prediction"] = 'none'

for i in df.index:
    
    digit_list = spokentonumer.del_non_digit(df.loc[i, '年龄文本'])
    
    if (len(digit_list) > 0):
            
            clasn = digit_list[0]
            if int(clasn) == 1:
                clas = "1岁以下"
            elif int(clasn) in range(2,18):
                clas = "1-17岁"
            elif int(clasn) in range(18,66):
                clas = "18-65岁"
            elif int(clasn) in range(66,120):
                clas = ">65岁"
            else:
                clas = 'none'
    else:
        clas = 'none'
        
    for word in ["elder", "elderly", "older", "aging"]:
        if word in df.loc[i, '年龄文本']:
            clas = ">65岁"
    for word in ["adult", "adults", "middle","mature"]:
        if word in df.loc[i, '年龄文本']:
            clas = "18-65岁"
    for word in ["young", "younger", "child", "children","adolescent", "adolescents", "paediatric", "pediatric"]:
        if word in df.loc[i, '年龄文本']:
            clas = "1-17岁"
    for word in ["infant", "infants", "neonatus", "neonate","neonatal","day", "days", "month", "months", "months","premature"]:
        if word in df.loc[i, '年龄文本']:
            clas = "1岁以下"
            
    df.loc[i, "prediction"] = clas

['forty', 'two']
['fifty', 'six']
['sixty', 'five']


In [20]:
eval.multiclass_score(df[["golden"]], df[["prediction"]])

结果: 0.7838827838827839 0.8337988694430343 0.7838827838827839 0.7760572288517577


(0.7838827838827839,
 0.8337988694430343,
 0.7838827838827839,
 0.7760572288517577)

# 患者性别

In [42]:
df = pd.read_excel("/home/data/t070224/Pubmed/Pubmed_Label/3.Orgnize/output/文献合并/human_label_pat.xlsx")
df = df.loc[df["是否是致病菌"]=='yes', :]
df = df.fillna("none")
df['golden'] = df['患者性别']
df["prediction"] = 'none'

for i in df.index:
    
    dig_list = df.loc[i, '患者性别文本'].lower().split(" ")
    for dig in dig_list:
        if dig in ["man", "men", "boy", "boys","male", "males", 
                    "he", "gentleman", "gentlemen", "his", "him"]:
            clas = "男"
            
        if dig in ["woman", "women", "girl","girls", "female", 
                    "females", "fm", "she", "lady", "ladys", "her"]:
            clas = "女"
            
    else:
        clas = "none"

In [43]:
eval.multiclass_score(df[["golden"]], df[["prediction"]])

结果: 0.5421245421245421 0.29389901917374445 0.5421245421245421 0.381161198263336


/home/data/t070224/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.5421245421245421,
 0.29389901917374445,
 0.5421245421245421,
 0.381161198263336)

# 证据等级

In [58]:
df = pd.read_excel("/home/data/t070224/Pubmed/Pubmed_Label/3.Orgnize/output/文献合并/human_label_pat.xlsx")
df = df.loc[df["是否是致病菌"]=='yes', :]
df = df.fillna("none")
df['golden'] = df['证据等级']
df["prediction"] = 'none'

for i in df.index:
    
    dig_list = df.loc[i, 'Full'].lower().split(" ")
    
    for dig in dig_list:
        if dig in [
            "autopsy", "biopsy", "biopsies", 
            "tissue", 
            "histopathology", "histology", "histological", "histopathological"]:
            clas = "1：同时具备肺组织病理+病原学证据"
        if dig in [
            "blood",
            "pleural", "purulent",
            "csf","cerebrospinal"]:
                clas = "3：血及无菌体液（如脓胸液）"
        if dig in [
            "bronchoscopy","bronchoscopies", "tracheal", "bronchial", "throat",
            "bal", "balf", "bronchoalveolar","lavage"]:
                clas ="4：气管镜活检组织培养出病原体，或者支气管肺泡灌洗液（BALF）定量培养>10**4"
        if dig in [
            "aspirate", "aspiration", 
            "secretion", "sputum", "sputa","immunohistochemistry","staining"]:
                clas ="5：气管吸出物或痰/BALF纯培养或优势培养出病原体"
    df.loc[i, "prediction"] = clas

In [59]:
eval.multiclass_score(df[["golden"]], df[["prediction"]])

结果: 0.2783882783882784 0.11999096615798058 0.2783882783882784 0.16607810118864885


/home/data/t070224/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.2783882783882784,
 0.11999096615798058,
 0.2783882783882784,
 0.16607810118864885)

In [46]:
df['golden'].value_counts()

none                                          311
3：血及无菌体液（如脓胸液）                                 86
5：气管吸出物或痰/BALF纯培养或优势培养出病原体                     67
4：气管镜活检组织培养出病原体，或者支气管肺泡灌洗液（BALF）定量培养>10**4     56
2：肺组织病原学证据                                     16
1：同时具备肺组织病理+病原学证据                              10
Name: golden, dtype: int64

In [60]:
df['prediction'].value_counts()

5：气管吸出物或痰/BALF纯培养或优势培养出病原体                    186
3：血及无菌体液（如脓胸液）                                174
4：气管镜活检组织培养出病原体，或者支气管肺泡灌洗液（BALF）定量培养>10**4    113
1：同时具备肺组织病理+病原学证据                              73
Name: prediction, dtype: int64